In [ ]:
from __future__ import annotations
from typing import TypedDict, List, Dict
import os, json, re, textwrap
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END

In [ ]:
load_dotenv("config.env")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
tavily = TavilySearchResults(k=8)
ALLOWED_DOMAINS = ("nih.gov","mayoclinic.org","who.int","cdc.gov","medlineplus.gov")

# --- utils ---
def hr(n=60): print("─"*n)
def title(s): hr(); print(s); hr()

# --- state ---
class HealthState(TypedDict, total=False):
    step: int
    topic: str
    results: List[Dict]
    sources: List[str]
    summary: str
    quiz_q: str
    quiz_choices: List[str]
    quiz_answer: str
    rationale: str
    patient_answer: str
    continue_flow: bool

def bump(s: HealthState, **updates) -> HealthState:
    """Return ONLY deltas + step increment (guarantees each node writes)."""
    return {"step": s.get("step", 0) + 1, **updates}

In [ ]:
# --- helpers ---
def filter_sources(results: List[Dict]) -> List[str]:
    urls = []
    for r in results:
        u = r.get("url","")
        if any(dom in u for dom in ALLOWED_DOMAINS): urls.append(u)
    seen, out = set(), []
    for u in urls:
        if u not in seen:
            out.append(u); seen.add(u)
            if len(out)>=5: break
    return out or [r.get("url","") for r in results[:5]]

def summarize_for_patient(topic: str, sources: List[str]) -> str:
    srcs = "\n".join(f"[{i+1}] {u}" for i,u in enumerate(sources))
    prompt = f"""You are a medical educator for patients (≈8th-grade level).
Summarize key points about: "{topic}" using only the sources below.
Write 5–8 short bullets with bracket citations [1],[2] that match the list.

Sources:
{srcs}
"""
    return llm.invoke(prompt).content.strip()

def make_quiz(summary: str) -> Dict:
    prompt = f"""From the summary below, create ONE multiple-choice question.
Return ONLY JSON:
{{"question": str, "choices": ["A. ...","B. ...","C. ...","D. ..."],
 "answer": "A|B|C|D", "rationale": "1–3 sentences with [1]/[2] citations"}}
Summary:
{summary}
JSON:"""
    raw = llm.invoke(prompt).content.strip()
    raw = re.sub(r"^```json|```$", "", raw, flags=re.IGNORECASE|re.MULTILINE).strip()
    return json.loads(raw)

def print_summary(summary: str, sources: List[str]):
    title("Patient-Friendly Summary")
    print(summary, "\nSources:"); 
    for i,u in enumerate(sources,1): print(f"[{i}] {u}")
    hr()

def print_quiz(q: str, choices: List[str]):
    title("Comprehension Check")
    print(q+"\n"); [print(c) for c in choices]; hr()

def grade_answer(user: str, correct: str, rationale: str) -> Dict[str,str]:
    u, c = (user or "").strip().upper(), correct.strip().upper()
    if u == c: 
        return {"grade":"A (Correct)","feedback":f"Nice work! {rationale}"}
    return {"grade":"C (Try again)","feedback":f"Expected {c}. {rationale}"}

In [ ]:
# --- nodes (return deltas via bump) ---
def ask_topic(s: HealthState) -> HealthState:
    title("Welcome to HealthBot")
    print("Educational only, not medical advice.")
    topic = input("\nWhat health topic/condition would you like to learn about? ").strip()
    return bump(s, topic=topic)

def search_node(s: HealthState) -> HealthState:
    query = f"""{s['topic']} site:nih.gov OR site:mayoclinic.org OR site:who.int OR site:cdc.gov OR site:medlineplus.gov"""
    results = tavily.invoke(query)
    return bump(s, results=results, sources=filter_sources(results))

def summarize_node(s: HealthState) -> HealthState:
    return bump(s, summary=summarize_for_patient(s["topic"], s["sources"]))

def present_summary_node(s: HealthState) -> HealthState:
    print_summary(s["summary"], s["sources"])
    input("Press Enter when you're ready for a quick comprehension check… ")
    return bump(s, patient_answer="")  # clear any previous answer

def make_quiz_node(s: HealthState) -> HealthState:
    q = make_quiz(s["summary"])
    return bump(s, quiz_q=q["question"], quiz_choices=q["choices"],
                quiz_answer=q["answer"], rationale=q["rationale"])

def ask_quiz_node(s: HealthState) -> HealthState:
    print_quiz(s["quiz_q"], s["quiz_choices"])
    ans = input("Your answer (A/B/C/D): ").strip().upper()
    while ans not in {"A","B","C","D"}:
        ans = input("Please enter A, B, C, or D: ").strip().upper()
    return bump(s, patient_answer=ans)

def grade_node(s: HealthState) -> HealthState:
    r = grade_answer(s["patient_answer"], s["quiz_answer"], s["rationale"])
    title("Your Results"); print(f"Grade: {r['grade']}\n")
    print(textwrap.fill(r["feedback"], width=90)); hr()
    return bump(s, continue_flow=False)  # write a tracked key

def ask_continue_node(s: HealthState) -> HealthState:
    again = input("\nLearn another topic? (y/n): ").strip().lower()
    return bump(s, continue_flow=again.startswith("y"))

def reset_node(s: HealthState) -> HealthState:
    print("\nStarting a new session. Previous context cleared for privacy.")
    return bump(s, topic="", results=[], sources=[], summary="",
                quiz_q="", quiz_choices=[], quiz_answer="", rationale="",
                patient_answer="", continue_flow=False)

In [ ]:
# --- graph wiring ---
graph = StateGraph(HealthState)
graph.add_node("ask_topic", ask_topic)
graph.add_node("search", search_node)
graph.add_node("summarize", summarize_node)
graph.add_node("present_summary", present_summary_node)
graph.add_node("make_quiz", make_quiz_node)
graph.add_node("ask_quiz", ask_quiz_node)
graph.add_node("grade", grade_node)
graph.add_node("ask_continue", ask_continue_node)
graph.add_node("reset", reset_node)

graph.set_entry_point("ask_topic")
graph.add_edge("ask_topic", "search")
graph.add_edge("search", "summarize")
graph.add_edge("summarize", "present_summary")
graph.add_edge("present_summary", "make_quiz")
graph.add_edge("make_quiz", "ask_quiz")
graph.add_edge("ask_quiz", "grade")
graph.add_edge("grade", "ask_continue")
graph.add_conditional_edges(
    "ask_continue",
    lambda s: "restart" if s.get("continue_flow") else "exit",
    {"restart": "reset", "exit": END},
)
graph.add_edge("reset", "ask_topic")

app = graph.compile()

# --- run ---
_ = app.invoke(HealthState(step=0))


────────────────────────────────────────────────────────────
Welcome to HealthBot
────────────────────────────────────────────────────────────
Educational only, not medical advice.
────────────────────────────────────────────────────────────
Patient-Friendly Summary
────────────────────────────────────────────────────────────
- Diabetes is a disease that occurs when your blood sugar (glucose) levels are too high [1].
- There are two main types of diabetes: Type 1, where the body doesn't make insulin, and Type 2, where the body doesn't use insulin properly [1][2].
- Insulin is a hormone that helps sugar from food get into your cells for energy [2].
- Symptoms of diabetes can include increased thirst, frequent urination, extreme fatigue, and blurred vision [2].
- Managing diabetes involves monitoring blood sugar levels, eating a healthy diet, exercising, and sometimes taking medication [3][4].
- People with diabetes need to be careful about their foot care, as they can develop serious fo